1. Create a Jupyter Notebook using the advanced web scraping notebook Download advanced web scraping notebook  from Module 9 as a template.

2. Scrape the NJIT Course Schedule web page and create a pandas DataFrame comprising all sections of all courses offered by the Finance Department (FIN) for Fall 2025.

3. Using the DataFrame from step 2, create another DataFrame that computes:  Maximum Seats (Max), Seats Already Taken (Now) and Seats Available (derived) by instructor.

4. Submit the Jupyter notebook, properly named, for this assignment.

NOTE:  you will probably need to install the selenium library.  The easiest way is to run pip install selenium from a command prompt.  See me if you have any problems getting the first 3 cells of the notebook to execute correctly.

In [61]:
# uncomment the selenium installation line if necessary
%pip install selenium
import selenium
from selenium import webdriver
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: selenium in c:\users\arnav\appdata\roaming\python\python312\site-packages (4.26.1)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
driver = webdriver.Chrome()
#driver = webdriver.Firefox()  


In [63]:
driver.get('https://generalssb-prod.ec.njit.edu/BannerExtensibility/customPage/page/stuRegCrseSched')

In [64]:
driver.implicitly_wait(5)
left_link = driver.find_element(By.ID, 'pbid-subjListTableSubjectLink-32')

In [65]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Locate the FIN link by ID directly (no need for left_link intermediate step)
FIN_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "pbid-subjListTableSubjectLink-32"))
)

# Scroll into view and click using JS to avoid overlays
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", FIN_link)
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "pbid-subjListTableSubjectLink-32"))
)
driver.execute_script("arguments[0].click();", FIN_link)


In [66]:
my_span = driver.find_element(by=By.ID, value='pbid-courseListSectionDetailSections-0')

In [67]:
print(my_span)

<selenium.webdriver.remote.webelement.WebElement (session="09f2b02453d7c01e6997e5ce07dfccdd", element="f.EBB6B63375929CA9D907717FD03B7D5A.d.4FC644EEFE653C43723D166E1970043B.e.33")>


In [68]:
course_headers = my_span.find_elements(by=By.TAG_NAME, value="H4")

In [69]:
course_tables = my_span.find_elements(by=By.TAG_NAME, value="table")

In [70]:
print("I have {} headers and {} tables".format(len(course_headers), len(course_tables)))

I have 18 headers and 18 tables


In [71]:
def create_column_headings(table_headers):
    loc_columns = ['Course_name']
    for i in range(len(table_headers)):
        loc_columns.append(table_headers[i].text)
    return loc_columns

In [72]:
def create_course_listings(c_headers, c_tables):
    c_list = []
    for i in range(len(c_headers)):
        course_name = c_headers[i].text
#       collect the rows for each section of the course
        t_rows = c_tables[i].find_elements(by=By.CLASS_NAME, value='success')
#       for each row, get all of the cells
        for k in range(len(t_rows)):
            course_row = []
            course_row.append(course_name)
            t_cells = t_rows[k].find_elements(by=By.TAG_NAME, value='td')
            for l in range(len(t_cells)):
                course_row.append(t_cells[l].text)
            c_list.append(course_row)
    return c_list


In [73]:
# create column headings by using the 'th' cells of the first table
t_headers = course_tables[0].find_elements(by=By.TAG_NAME, value="th")
columns = create_column_headings(t_headers)

In [74]:
# use the list of course_headers and the list of course_tables to build
#   individual rows - one for each course/section
course_list = create_course_listings(course_headers, course_tables)
df = pd.DataFrame(course_list, columns=columns)
driver.quit()

In [75]:
df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,FIN 218 - FINANCL MARKETS & INSTITUTIONS,101,93392,W,6:00 PM - 8:50 PM,KUPF 107,Open,43,23,"Hammond, John",Face-to-Face,3,Book,
1,FIN 218 - FINANCL MARKETS & INSTITUTIONS - HONORS,HM1,93394,TR,10:00 AM - 11:20 AM,CKB G-08,Open,10,8,"Li, Jixing",Face-to-Face,3,Book,Honors Course
2,FIN 315 - FUND OF CORPORATE FINANCE,001,93397,WF,1:00 PM - 2:20 PM,CULM LECT 3,Open,33,19,"Uddin, Ajim",Face-to-Face,3,Book,
3,FIN 315 - FUND OF CORPORATE FINANCE,003,93398,MR,2:30 PM - 3:50 PM,KUPF 207,Open,43,41,"Fox, Wayne",Face-to-Face,3,Book,
4,FIN 315 - FUND OF CORPORATE FINANCE,101,93399,M,6:00 PM - 8:50 PM,CULM LECT 3,Open,43,23,"Fox, Wayne",Face-to-Face,3,Book,
5,FIN 315 - FUND OF CORPORATE FINANCE - HONORS,HM1,93402,WF,1:00 PM - 2:20 PM,CULM LECT 3,Open,10,3,"Uddin, Ajim",Face-to-Face,3,Book,Honors Course
6,FIN 403 - FINANCIAL STATEMENT ANALYSIS,451,93404,,TBA,,Open,35,30,"Athari, Mahtab",Online Newark,3,Book,Online Course
7,FIN 410 - DATA MINING & MACHINE LEARNING,001,93405,TR,4:00 PM - 5:20 PM,KUPF 203,Open,43,20,"Yu, Dantong",Face-to-Face,3,Book,Students must bring their own device for this ...
8,FIN 422 - INTERNATIONAL FINANCE,101,93407,M,6:00 PM - 8:50 PM,CKB 124,Open,43,34,"Domasky, Peter",Face-to-Face,3,Book,
9,FIN 600 - CORPORATE FINANCE I,001,93409,T,2:30 PM - 5:20 PM,CAB 3052,Open,43,8,"Fox, Wayne",Hybrid,3,Book,Hybrid Course - Meets Alternate Weeks\nFace-to...


In [76]:
df.describe()

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
count,20,20,20,20,20,20,20,20,20,20,20,20,20,20
unique,13,9,20,8,7,12,1,5,18,13,4,1,1,13
top,FIN 600 - CORPORATE FINANCE I,101,93392,,6:00 PM - 8:50 PM,,Open,43,23,"Uddin, Ajim",Face-to-Face,3,Book,
freq,6,7,1,6,8,6,20,11,2,3,8,20,20,5


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    20 non-null     object
 1   Section        20 non-null     object
 2   CRN            20 non-null     object
 3   Days           20 non-null     object
 4   Times          20 non-null     object
 5   Location       20 non-null     object
 6   Status         20 non-null     object
 7   Max            20 non-null     object
 8   Now            20 non-null     object
 9   Instructor     20 non-null     object
 10  Delivery Mode  20 non-null     object
 11  Credits        20 non-null     object
 12  Info           20 non-null     object
 13  Comments       20 non-null     object
dtypes: object(14)
memory usage: 2.3+ KB


In [78]:
df['Max'] = df['Max'].astype('int')
df['Now'] = df['Now'].astype('int')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    20 non-null     object
 1   Section        20 non-null     object
 2   CRN            20 non-null     object
 3   Days           20 non-null     object
 4   Times          20 non-null     object
 5   Location       20 non-null     object
 6   Status         20 non-null     object
 7   Max            20 non-null     int64 
 8   Now            20 non-null     int64 
 9   Instructor     20 non-null     object
 10  Delivery Mode  20 non-null     object
 11  Credits        20 non-null     object
 12  Info           20 non-null     object
 13  Comments       20 non-null     object
dtypes: int64(2), object(12)
memory usage: 2.3+ KB


In [80]:
df = df[~df['Course_name'].str.contains('CS 488')]
df = df[~df['Course_name'].str.contains('CS 700B')]
df = df[~df['Course_name'].str.contains('CS 701B')]
df = df[~df['Course_name'].str.contains('CS 725')]
df = df[~df['Course_name'].str.contains('CS 792')]
df = df[~df['Course_name'].str.contains('CS 790A')]
df = df[~df['Course_name'].str.contains('CS 726')]

df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,FIN 218 - FINANCL MARKETS & INSTITUTIONS,101,93392,W,6:00 PM - 8:50 PM,KUPF 107,Open,43,23,"Hammond, John",Face-to-Face,3,Book,
1,FIN 218 - FINANCL MARKETS & INSTITUTIONS - HONORS,HM1,93394,TR,10:00 AM - 11:20 AM,CKB G-08,Open,10,8,"Li, Jixing",Face-to-Face,3,Book,Honors Course
2,FIN 315 - FUND OF CORPORATE FINANCE,001,93397,WF,1:00 PM - 2:20 PM,CULM LECT 3,Open,33,19,"Uddin, Ajim",Face-to-Face,3,Book,
3,FIN 315 - FUND OF CORPORATE FINANCE,003,93398,MR,2:30 PM - 3:50 PM,KUPF 207,Open,43,41,"Fox, Wayne",Face-to-Face,3,Book,
4,FIN 315 - FUND OF CORPORATE FINANCE,101,93399,M,6:00 PM - 8:50 PM,CULM LECT 3,Open,43,23,"Fox, Wayne",Face-to-Face,3,Book,
5,FIN 315 - FUND OF CORPORATE FINANCE - HONORS,HM1,93402,WF,1:00 PM - 2:20 PM,CULM LECT 3,Open,10,3,"Uddin, Ajim",Face-to-Face,3,Book,Honors Course
6,FIN 403 - FINANCIAL STATEMENT ANALYSIS,451,93404,,TBA,,Open,35,30,"Athari, Mahtab",Online Newark,3,Book,Online Course
7,FIN 410 - DATA MINING & MACHINE LEARNING,001,93405,TR,4:00 PM - 5:20 PM,KUPF 203,Open,43,20,"Yu, Dantong",Face-to-Face,3,Book,Students must bring their own device for this ...
8,FIN 422 - INTERNATIONAL FINANCE,101,93407,M,6:00 PM - 8:50 PM,CKB 124,Open,43,34,"Domasky, Peter",Face-to-Face,3,Book,
9,FIN 600 - CORPORATE FINANCE I,001,93409,T,2:30 PM - 5:20 PM,CAB 3052,Open,43,8,"Fox, Wayne",Hybrid,3,Book,Hybrid Course - Meets Alternate Weeks\nFace-to...


In [81]:
# Group by instructor and summarize seat data
instructor_summary = df.groupby('Instructor').agg(
    Max_Seats=('Max', 'sum'),
    Seats_Taken=('Now', 'sum')
).reset_index()

# Now compute Seats_Available as a derived column
instructor_summary['Seats_Available'] = instructor_summary['Max_Seats'] - instructor_summary['Seats_Taken']

# Sort by available seats
instructor_summary.sort_values(by='Seats_Available', ascending=False)


,Instructor,Max_Seats,Seats_Taken,Seats_Available
4,"Fox, Wayne",129,72,57
3,"Eliopoulos, George",73,18,55
10,"Uddin, Ajim",86,32,54
11,"Xu, Yanlong",43,2,41
5,"Goodwin, Kenneth",60,25,35
8,"Liu, Yanguang",43,9,34
9,"Tao, Xinyuan",73,41,32
2,"Ehrlich, Michael",30,5,25
12,"Yu, Dantong",43,20,23
6,"Hammond, John",43,23,20
